In [2]:
import pandas as pd
import requests
from io import StringIO

from nst_scraper import nst_on_ice_scraper, nst_team_on_ice_scraper

pd.set_option('display.max_columns', None)


In [5]:
nst_team_on_ice_scraper(startdate='', enddate='2024-10-12',last_n=30)

,team,gp,toi,w,l,otl,row,points,point_%,cf,ca,cf%,ff,fa,ff%,sf,sa,sf%,gf,ga,gf%,xgf,xga,xgf%,scf,sca,scf%,scsf,scsa,scsf%,scgf,scga,scgf%,scsh%,scsv%,hdcf,hdca,hdcf%,hdsf,hdsa,hdsf%,hdgf,hdga,hdgf%,hdsh%,hdsv%,mdcf,mdca,mdcf%,mdsf,mdsa,mdsf%,mdgf,mdga,mdgf%,mdsh%,mdsv%,ldcf,ldca,ldcf%,ldsf,ldsa,ldsf%,ldgf,ldga,ldgf%,ldsh%,ldsv%,sh%,sv%,pdo
0,Ottawa Senators,14,850.68,8,6,0,6,16,0.571,874,823,51.50,632,571,52.54,412,393,51.18,32,37,46.38,43.44,41.28,51.27,404,394,50.63,211,210,50.12,23,29,44.23,10.90,86.19,135,171,44.12,88,113,43.78,14,17,45.16,15.91,84.96,269,223,54.67,123,97,55.91,9,12,42.86,7.32,87.63,525,454,53.63,177,160,52.52,8,8,50.00,4.52,95.00,7.77,90.59,0.984
1,Buffalo Sabres,14,843.97,6,7,1,6,13,0.464,889,817,52.11,626,605,50.85,408,409,49.94,38,40,48.72,46.59,47.21,49.67,417,404,50.79,207,222,48.25,31,34,47.69,14.98,84.68,154,176,46.67,95,112,45.89,19,19,50.00,20.00,83.04,263,228,53.56,112,110,50.45,12,15,44.44,10.71,86.36,511,496,50.74,184,167,52.42,4,6,40.00,2.17,96.41,9.31,90.22,0.995
2,Boston Bruins,12,730.20,7,5,0,6,14,0.583,623,757,45.14,451,525,46.21,329,337,49.40,33,32,50.77,33.17,42.98,43.56,302,384,44.02,169,185,47.74,25,30,45.45,14.79,83.78,131,164,44.41,89,107,45.41,17,25,40.48,19.10,76.64,171,220,43.73,80,78,50.63,8,5,61.54,10.00,93.59,350,414,45.81,131,135,49.25,6,1,85.71,4.58,99.26,10.03,90.50,1.005
3,Tampa Bay Lightning,12,725.00,7,4,1,7,15,0.625,726,735,49.69,522,520,50.10,382,357,51.69,45,33,57.69,39.25,38.31,50.61,360,351,50.63,197,190,50.90,30,27,52.63,15.23,85.79,146,137,51.59,95,93,50.53,18,16,52.94,18.95,82.80,214,214,50.00,102,97,51.26,12,11,52.17,11.76,88.66,413,429,49.05,159,147,51.96,13,6,68.42,8.18,95.92,11.78,90.76,1.025
4,Detroit Red Wings,12,736.93,6,4,2,5,14,0.583,713,809,46.85,514,569,47.46,370,393,48.49,38,39,49.35,36.68,39.52,48.14,326,392,45.40,180,202,47.12,28,34,45.16,15.56,83.17,116,151,43.45,87,93,48.33,15,19,44.12,17.24,79.57,210,241,46.56,93,109,46.04,13,15,46.43,13.98,86.24,432,493,46.70,164,177,48.09,10,5,66.67,6.10,97.18,10.27,90.08,1.003
5,Carolina Hurricanes,10,600.00,7,3,0,7,14,0.700,711,438,61.88,529,326,61.87,337,235,58.92,32,21,60.38,42.51,22.64,65.26,346,196,63.84,177,114,60.82,27,16,62.79,15.25,85.96,147,87,62.82,95,63,60.13,14,10,58.33,14.74,84.13,199,109,64.61,82,51,61.65,13,6,68.42,15.85,88.24,430,261,62.23,152,103,59.61,5,3,62.50,3.29,97.09,9.50,91.06,1.006
6,New York Islanders,13,789.98,9,2,2,9,20,0.769,786,868,47.52,580,573,50.30,386,400,49.11,38,31,55.07,41.87,38.37,52.18,389,401,49.24,211,204,50.84,30,25,54.55,14.22,87.75,162,162,50.00,109,110,49.77,17,15,53.13,15.60,86.36,227,239,48.71,102,94,52.04,13,10,56.52,12.75,89.36,467,509,47.85,158,172,47.88,7,4,63.64,4.43,97.67,9.84,92.25,1.021
7,Florida Panthers,13,791.05,7,5,1,6,15,0.577,935,655,58.81,642,512,55.63,449,358,55.64,42,35,54.55,47.41,36.64,56.40,434,303,58.89,234,183,56.12,32,27,54.24,13.68,85.25,193,131,59.57,125,89,58.41,18,16,52.94,14.40,82.02,241,172,58.35,109,94,53.69,14,11,56.00,12.84,88.30,552,403,57.80,191,147,56.51,9,6,60.00,4.71,95.92,9.35,90.22,0.996
8,Philadelphia Flyers,11,666.40,3,7,1,2,7,0.318,724,639,53.12,520,416,55.56,350,271,56.36,23,43,34.85,34.45,33.84,50.45,327,286,53.34,163,140,53.80,17,34,33.33,10.43,75.71,146,109,57.25,86,77,52.76,8,20,28.57,9.30,74.03,181,177,50.56,77,63,55.00,9,14,39.13,11.69,77.78,454,371,55.03,164,112,59.42,6,8,42.86,3.66,92.86,6.57,84.13,0.907
9,Montreal Canadiens,14,855.87,5,5,4,5,14,0.500,743,1030,41.91,526,726,42.01,365,499,42.25,49,49,50.00,39.98,55.49,41.88,366,501,42.21,209,267,43.91,41,37,52.56,19.62,86.14,148,210,41.34,108,140,43.55,31,20,60.78,28.70,85.71,218,291,42.83,101,127,44.30,10,17,37.04,9.90,86.61,399,594,40.18,139,212,39.60,8,12,40.00,5.76,94.34,13.42,90.18,1.036


In [3]:
df = nst_on_ice_scraper(startdate='', enddate='2024-10-09', last_n=30, rate='y', lines='single')
df


,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,"DAL, STL",D,12,179.27,14.94,0.00,0.33,0.33,0.00,0.33,16.67,5.02,0.00,0.18,10.71,8.03,4.02,0.00,0.00,0.33,0.67,0.33,0.33,0.00,0.0,0.00,0.00,0.00,3.01,3.35,3.35,0.00,0.00,-
1,Jeff Carter,PIT,C,11,101.63,9.24,0.00,0.00,0.00,0.00,0.00,0.00,4.72,0.00,0.31,8.86,7.08,5.90,1.77,0.00,1.77,0.00,0.00,0.00,0.00,0.0,0.00,0.59,0.00,5.90,2.36,2.36,33.65,20.66,36.58
2,Zach Parise,COL,L,12,129.63,10.80,0.46,0.00,0.00,0.00,0.46,50.00,5.09,9.09,0.66,8.33,6.02,5.09,3.70,0.46,0.93,1.85,0.93,0.93,0.00,0.0,0.46,0.46,1.85,2.78,1.39,4.63,1.39,5.09,9.92
3,Brent Burns,CAR,D,11,165.50,15.05,0.00,0.36,0.00,0.36,0.36,25.00,6.89,0.00,0.20,19.58,11.60,3.99,0.36,0.00,1.81,1.45,0.73,0.73,0.00,0.0,0.36,2.90,1.81,1.81,0.36,3.63,0.00,0.00,-
4,Corey Perry,EDM,R,15,179.98,12.00,0.67,0.67,0.33,0.33,1.33,44.44,6.33,10.53,0.79,12.00,10.00,6.67,2.67,0.67,0.67,3.00,1.00,0.67,0.33,0.0,1.00,0.67,2.00,3.00,7.33,1.67,0.00,0.00,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,Calum Ritchie,COL,C,1,12.53,12.53,0.00,0.00,0.00,0.00,0.00,-,4.79,0.00,1.62,14.36,14.36,9.57,4.79,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,9.57,0.00,4.79,0.00,4.79,0.00
773,Sam Malinski,COL,D,1,12.92,12.92,0.00,0.00,0.00,0.00,0.00,-,4.65,0.00,0.03,4.65,4.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,4.65,4.65,0.00,0.00,9.29,0.00,0.00,-
774,Uvis Balinskis,FLA,D,7,88.90,12.70,0.00,0.67,0.00,0.67,0.67,50.00,2.02,0.00,0.45,6.07,4.05,2.70,1.35,0.00,1.35,2.70,1.35,1.35,0.00,0.0,2.02,2.02,1.35,3.37,12.82,1.35,0.00,0.00,-
775,Jiri Smejkal,OTT,L,8,74.93,9.37,0.80,0.00,0.00,0.00,0.80,100.00,1.60,50.00,0.34,5.60,4.80,3.20,1.60,0.00,0.00,1.60,0.80,0.80,0.00,0.0,0.80,3.20,1.60,11.21,4.80,4.00,6.41,10.41,30.50


In [ ]:
df = nst_team_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='2024-10-09', last_n=30, rate='y', lines='single')
